## initial code for our Hand Gesture Recognition

In [10]:
# ML models imports
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from skimage.feature import hog

# destribution models imports
from scipy.stats import randint, uniform

# Data manipulation imports
import numpy as np
import pandas as pd
import tqdm as tqdm

# visualisation models imports
import matplotlib.pyplot as plt

# image processing imports
import skimage.io as io
import cv2
from skimage.transform import resize

# dealing with files
import os

# visual dataset (to test randomized gridsearch not needed for now)
from sklearn.datasets import make_hastie_10_2  # to test our models

# from utils import prepareData, LoadData, FeatureExtraction, preprocess

In [11]:
def segment(image):
    blured_image = cv2.GaussianBlur(image, (7, 7), 0)
    ycbcr_image = cv2.cvtColor(blured_image, cv2.COLOR_BGR2YCrCb)
    # Extract the Cr channel
    cr_channel = ycbcr_image[:,:,1]

    # Apply thresholding to obtain a binary image
    _, binary_img = cv2.threshold(cr_channel,0,255,cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Define the structuring element for the closing operation
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (25, 25))

    # Perform the closing operation
    closed_img = cv2.morphologyEx(binary_img, cv2.MORPH_CLOSE, kernel)

    # Find the contours in the binary image
    contours, hierarchy = cv2.findContours(closed_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filling the contours on a copy of the original image
    img_contours = cv2.cvtColor(cr_channel, cv2.COLOR_GRAY2BGR)
    cv2.drawContours(img_contours, contours, -1, (0, 0, 0), -1)

    return img_contours

In [12]:
def FeatureExtraction(image):
    # this written code is an initial code for extracting features

    '''

        TODO: Feature Extraction code should be implemented here.  

    '''
    
    resized_image = resize(image,(1000,1000))   # downscaing from approx 2500x4000 to 500x500
    
    # Extract the hog features
    # block_norm uses L2 norm with hysterisis for reducing effect of illuminacity
    # transform_sqrt for applying gamma correction
    segmented = segment(image)

    hog_features = hog(segmented, block_norm='L2-Hys', feature_vector=True, transform_sqrt=True, channel_axis=2)

    # image = np.array(resized).flatten() # flatten our image to be used as input vector to our model

    return hog_features

In [15]:
def LoadData():
    Features=[]
    labels=[]

    i = 0

    for gender in ["men","Women"]:
        datadir = r"Dataset\{}".format(gender)
        # loop over gender
        for hand in os.listdir(datadir): 
            # loop over each class [0,1,2,3,4,5]
            for img in os.listdir(datadir+ "/" +str(hand)):
                # ignoring anything except images
                if((img.split('.')[-1]).lower() not in ['jpg','png','jpeg']):
                    continue

                # loading our images
                img_array=io.imread(datadir + "/" + str(hand) + "/" + img )  # approx 2500 * 4000

                # append extracted features to Featurees list           
                Features.append(FeatureExtraction(img_array)) 

                # append class of image.
                labels.append(hand)  

                print(f'image Number: {i}')
                i+=1

    return np.asarray(Features),np.asarray(labels)

In [16]:
Features, labels = LoadData() 

image Number: 0
image Number: 1
image Number: 2
image Number: 3
image Number: 4
image Number: 5
image Number: 6
image Number: 7
image Number: 8
image Number: 9
image Number: 10
image Number: 11
image Number: 12
image Number: 13
image Number: 14
image Number: 15
image Number: 16
image Number: 17
image Number: 18
image Number: 19
image Number: 20
image Number: 21
image Number: 22
image Number: 23
image Number: 24
image Number: 25
image Number: 26
image Number: 27
image Number: 28
image Number: 29
image Number: 30
image Number: 31
image Number: 32
image Number: 33
image Number: 34
image Number: 35
image Number: 36
image Number: 37
image Number: 38
image Number: 39
image Number: 40
image Number: 41
image Number: 42
image Number: 43
image Number: 44
image Number: 45
image Number: 46
image Number: 47
image Number: 48
image Number: 49
image Number: 50
image Number: 51
image Number: 52
image Number: 53
image Number: 54
image Number: 55
image Number: 56
image Number: 57
image Number: 58
image N

MemoryError: Unable to allocate 91.1 MiB for an array with shape (2592, 4608) and data type float64

In [ ]:
np.save('labels.npy', labels)
np.save('features.npy', Features)

### to load our extracted features

In [36]:
# test = np.load('labels.npy')

In [29]:
#! pip install imageio[pyav] (in case of having error while loading some of images try this one)
labels

array(['0', '0', '0', '0', '0', '1', '1', '1', '1', '2', '2', '2', '2',
       '2', '3', '3', '3', '3', '3', '4', '4', '4', '4', '5', '5', '5',
       '5', '5', '0', '0', '0', '0', '0', '1', '1', '1', '1', '1', '2',
       '2', '2', '2', '2', '3', '3', '3', '3', '3', '4', '4', '4', '4',
       '4', '5', '5', '5', '5', '5'], dtype='<U1')

* # Selecting the best model

- ### Define hyperparameter grids for each model

In [30]:
param_distributions = {
    'RandomForestClassifier': {
        'n_estimators': randint(50, 500),
        'max_depth': randint(2, 20),
        'min_samples_split': randint(2, 10),
        'min_samples_leaf': randint(1, 5),
        'max_features': ['sqrt', 'log2']
    },
    'GradientBoostingClassifier': {
        'learning_rate': uniform(0.01, 0.2),
        'n_estimators': randint(50, 500),
        'max_depth': randint(2, 20),
        'min_samples_split': randint(2, 10),
        'min_samples_leaf': randint(1, 5),
        'max_features': ['sqrt', 'log2']
    },
    'SVC': {
        'C': uniform(0.01, 10),
        'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
        'degree': randint(2, 5),
        'gamma': ['scale', 'auto'] + list(np.arange(0.1, 1, 0.1))
    },
    'LogisticRegression': {
        'C': uniform(0.01, 10),
        'penalty': ['l1', 'l2', 'elasticnet', 'none'],
        'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
        'max_iter': randint(50, 500)
    },
    'DecisionTreeClassifier': {
        'criterion': ['gini', 'entropy'],
        'splitter': ['best', 'random'],
        'max_depth': randint(2, 20),
        'min_samples_split': randint(2, 10),
        'min_samples_leaf': randint(1, 5),
        'max_features': ['sqrt', 'log2']
    },
    'KNeighborsClassifier': {
        'n_neighbors': randint(3, 30),
        'weights': ['uniform', 'distance'],
        'algorithm': ['ball_tree', 'kd_tree', 'brute'],
        'leaf_size': randint(10, 100)
    },
    'GaussianNB': {
        'var_smoothing': uniform(1e-09, 1e-07)
    },
    'MLPClassifier': {
        'hidden_layer_sizes': [(50, 50), (100,), (100, 50)],
        'activation': ['identity', 'logistic', 'tanh', 'relu'],
        'solver': ['lbfgs', 'sgd', 'adam'],
        'alpha': uniform(0.0001, 0.01),
        'max_iter': randint(100, 1000)
    }
}

- ### Create a list of models to train (as example)

In [31]:
models = [
    RandomForestClassifier(),
    SVC(),
    GaussianNB()
]

- ### Loop over the models and fit  

In [32]:
# load our dummy data to test the randomizedSearche function
x,y = make_hastie_10_2()
df = pd.DataFrame(x)
df['Y'] = y

train, test = train_test_split(df, test_size=0.2) # this function shuffles the data points, and splits the data into
                                                  # 80% training set and 20% test set (indicated by test_size=0.2)
X_train, Y_train = train.iloc[:, :-1], train.iloc[:, -1]
X_test, Y_test = test.iloc[:, :-1], test.iloc[:, -1]
# Fit a simple decision tree first

In [33]:
for i, model in enumerate(models):
    print(f'Training Model {i+1}/{len(models)}: {str(model)[:-2]}')
    # Define randomized grid search
    random_search = RandomizedSearchCV(model, param_distributions[str(model)[:-2]], n_iter=10,cv=5, n_jobs=-1) # n_jobs means number of jobs to run in parallel. None means 1,
                                                                                                                # -1 means using all processors 😈.
    # Fit the randomized grid search to the data
    random_search.fit(X_train, Y_train)
    print(f'Best score: {random_search.best_score_:.3f}')
    print(f'Best parameters: {random_search.best_params_}\n')

    with open("analysis.txt", "a") as file:
        file.write(f'Training Model {i+1}/{len(models)}: {str(model)[:-2]})
        print(f'Best score: {random_search.best_score_:.3f}')
        print(f'Best parameters: {random_search.best_params_}\n')
        file.write("\n\n\n")

Training Model 1/3: RandomForestClassifier
Best score: 0.881
Best parameters: {'max_depth': 19, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 4, 'n_estimators': 179}

Training Model 2/3: SVC
Best score: 0.963
Best parameters: {'C': 5.8817763888073795, 'degree': 4, 'gamma': 0.4, 'kernel': 'poly'}

Training Model 3/3: GaussianNB
Best score: 0.979
Best parameters: {'var_smoothing': 1.066596175409692e-08}

